In [1]:
# Paso 1: Configuración del entorno

# Instalar las librerías necesarias
# Se recomienda usar un entorno virtual para evitar conflictos de dependencias
# Puedes crear un entorno virtual con `python -m venv nombre_entorno` y activarlo antes de instalar las librerías

# Instalación de librerías
!pip install rarfile pdfplumber spacy transformers pandas

# Descargar el modelo de spaCy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
import rarfile
import os
import pdfplumber
import re
import spacy
import json
from transformers import pipeline
import pandas as pd

In [6]:
# Cargar modelo de spaCy para el reconocimiento de entidades
nlp = spacy.load('en_core_web_sm')

# Pipeline para preguntas y respuestas (puede ser un modelo general)
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Ruta del archivo RAR
rar_path = '/content/Nueva carpeta.rar'  # Reemplaza con la ruta correcta

# Directorio para extraer los archivos
extraction_path = 'extracted_cvs'
os.makedirs(extraction_path, exist_ok=True)

def extract_rar(rar_path, extraction_path):
    """
    Extrae archivos de un archivo RAR a un directorio especificado.

    Args:
        rar_path (str): Ruta del archivo RAR.
        extraction_path (str): Directorio donde se extraerán los archivos.

    Returns:
        list: Lista de rutas de archivos extraídos.
    """
    with rarfile.RarFile(rar_path) as rf:
        rf.extractall(extraction_path)

    # Listar todos los archivos extraídos, incluyendo subcarpetas
    extracted_files = []
    for root, dirs, files in os.walk(extraction_path):
        for file in files:
            extracted_files.append(os.path.join(root, file))

    return extracted_files

In [7]:
# Extraer los archivos
cv_files = extract_rar(rar_path, extraction_path)
print(f"Archivos extraídos: {cv_files}")

def read_pdf(file_path):
    """
    Lee y extrae el texto de un archivo PDF.

    Args:
        file_path (str): Ruta del archivo PDF.

    Returns:
        str: Texto extraído del archivo PDF.
    """
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:  # Asegura que haya texto antes de añadir
                    text += page_text + "\n"
        return text
    except Exception as e:
        print(f"Error al leer {file_path}: {e}")
        return ""

Archivos extraídos: ['extracted_cvs/Nueva carpeta/1724342728_CV8.pdf', 'extracted_cvs/Nueva carpeta/1724342738_CV9.pdf', 'extracted_cvs/Nueva carpeta/1724342683_CV4.pdf', 'extracted_cvs/Nueva carpeta/1724342661_CV2.pdf', 'extracted_cvs/Nueva carpeta/1724342714_CV7.pdf', 'extracted_cvs/Nueva carpeta/1724342646_CV1.pdf', 'extracted_cvs/Nueva carpeta/1724342673_CV3.pdf', 'extracted_cvs/Nueva carpeta/1724342694_CV5.pdf', 'extracted_cvs/Nueva carpeta/1724342704_CV6.pdf']


In [8]:
# Función para extraer el nombre del candidato
def extract_name(text):
    """Extrae el nombre de una persona del texto utilizando spaCy."""
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return {"value": ent.text, "score": 0.9}
    return {"value": None, "score": 0}

# Función para extraer email o teléfono
def extract_contact(text):
    """Extrae el email o número de teléfono del texto."""
    email = re.findall(r'\S+@\S+', text)
    phone = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)
    if email:
        return {"value": email[0], "score": 0.9}
    elif phone:
        return {"value": phone[0], "score": 0.9}
    else:
        return {"value": None, "score": 0}

# Función para extraer los años de experiencia
def extract_experience(text):
    """Estima los años de experiencia profesional a partir del texto."""
    try:
        result = qa_pipeline({
            'context': text,
            'question': "How many years of experience does the candidate have?"
        })
        return {"value": result['answer'], "score": result['score']}
    except:
        return {"value": None, "score": 0}

# Función para verificar si hay formación en IA
def check_ai_education(text):
    """Determina si el candidato tiene formación en IA."""
    keywords = ["artificial intelligence", "machine learning", "deep learning"]
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return {"value": "S", "score": 0.9}
    return {"value": "N", "score": 0.9}

# Función principal para procesar el CV y extraer la información
def process_cv(cv_text):
    """Procesa el texto de un CV y extrae la información especificada."""
    return {
        "name": extract_name(cv_text),  # Nombre completo del candidato
        "contact": extract_contact(cv_text),  # Email o teléfono de contacto
        "experience_years": extract_experience(cv_text),  # Número de años de experiencia
        "ai_education": check_ai_education(cv_text)  # Formación en IA (S/N)
    }

# Procesar cada archivo de CV
results = []

for cv_path in cv_files:
    if cv_path.endswith('.pdf'):
        cv_text = read_pdf(cv_path)
        if cv_text:  # Si se leyó correctamente
            result = process_cv(cv_text)
            results.append(result)
            print(f"Procesado: {cv_path}")

# Convertir los resultados a JSON y mostrar
results_json = json.dumps(results, indent=4)
print(results_json)

# Guardar el resultado en un archivo JSON
with open('results.json', 'w') as json_file:
    json_file.write(results_json)

Procesado: extracted_cvs/Nueva carpeta/1724342728_CV8.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342738_CV9.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342683_CV4.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342661_CV2.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342714_CV7.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342646_CV1.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342673_CV3.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342694_CV5.pdf
Procesado: extracted_cvs/Nueva carpeta/1724342704_CV6.pdf
[
    {
        "name": {
            "value": "Law",
            "score": 0.9
        },
        "contact": {
            "value": "drsantoshkakade@gmail.com,",
            "score": 0.9
        },
        "experience_years": {
            "value": "20",
            "score": 0.4254302382469177
        },
        "ai_education": {
            "value": "N",
            "score": 0.9
        }
    },
    {
        "name": {
            "value": "W. Fullerton Pkwy",
            

In [10]:
# Cargar los datos desde el JSON
data = pd.read_json('resultss.json')

# Mostrar el contenido en un DataFrame para una visualización más clara
display(data)

,name,contact,experience_years,ai_education
0,"{'value': 'Law', 'score': 0.9}","{'value': 'drsantoshkakade@gmail.com,', 'score...","{'value': '20', 'score': 0.42543023824691706}","{'value': 'N', 'score': 0.9}"
1,"{'value': 'W. Fullerton Pkwy', 'score': 0.9}","{'value': 'loren@shevitz.org', 'score': 0.9}","{'value': '2003', 'score': 0.000571033218875}","{'value': 'N', 'score': 0.9}"
2,"{'value': 'Jawa Timur Dyah', 'score': 0.9}","{'value': 'dyahhediyati@gmail.com', 'score': 0.9}","{'value': '2018', 'score': 0.155094340443611}","{'value': 'N', 'score': 0.9}"
3,"{'value': 'POWELL', 'score': 0.9}","{'value': 'hello@reallygreatsite.com', 'score'...","{'value': '5 to 7', 'score': 0.284851163625717}","{'value': 'N', 'score': 0.9}"
4,"{'value': 'Cahyo Dwiputra', 'score': 0.9}","{'value': 'ringgicahyo@gmail.com', 'score': 0.9}","{'value': '12', 'score': 0.497433364391326}","{'value': 'N', 'score': 0.9}"
5,"{'value': 'Jawa Timur', 'score': 0.9}","{'value': 'me@junae.id', 'score': 0.9}","{'value': 'more than 6 years', 'score': 0.2809...","{'value': 'S', 'score': 0.9}"
6,"{'value': 'Michael Smith', 'score': 0.9}","{'value': None, 'score': 0}","{'value': '10+ years', 'score': 0.516781091690...","{'value': 'N', 'score': 0.9}"
7,"{'value': 'KENISAH', 'score': 0.9}","{'value': 'dhikayudano@gmail.com', 'score': 0.9}","{'value': '2021', 'score': 0.14425550401210702}","{'value': 'N', 'score': 0.9}"
8,"{'value': 'Alice Clark', 'score': 0.9}","{'value': None, 'score': 0}","{'value': '20+ years', 'score': 0.574561655521...","{'value': 'S', 'score': 0.9}"
